In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [4]:
data = requests.get(standings_url)

In [5]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[1]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [6]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
data = requests.get(team_urls[0])

In [8]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [9]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [10]:
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [12]:
shooting.head()

For Manchester City                                                     \
                 Date   Time              Comp                Round  Day   
0          2022-07-30  17:00  Community Shield  FA Community Shield  Sat   
1          2022-08-07  16:30    Premier League          Matchweek 1  Sun   
2          2022-08-13  15:00    Premier League          Matchweek 2  Sat   
3          2022-08-21  16:30    Premier League          Matchweek 3  Sun   
4          2022-08-27  15:00    Premier League          Matchweek 4  Sat   

                                         ... Standard               Expected  \
     Venue Result GF GA        Opponent  ...     Dist   FK PK PKatt       xG   
0  Neutral      L  1  3       Liverpool  ...      NaN  NaN  0     0      NaN   
1     Away      W  2  0        West Ham  ...     18.7  1.0  1     1      2.2   
2     Home      W  4  0     Bournemouth  ...     17.5  0.0  0     0      1.7   
3     Away      D  3  3   Newcastle Utd  ...     16.2  1.0  0     0      2.1   
4     Home      W  4  2  Crystal Palace  ...     14.1  0.0  0     0      2.2   

                            Unnamed: 25_level_0  
  npxG npxG/Sh G-xG np:G-xG        Match Report  
0  NaN     NaN  NaN     NaN        Match Report  
1  1.4    0.11 -0.2    -0.4        Match Report  
2  1.7    0.09  1.3     1.3        Match Report  
3  2.1    0.10  0.9     0.9        Match Report  
4  2.2    0.13  1.8     1.8        Match Report  

[5 rows x 26 columns]

In [13]:
shooting.columns = shooting.columns.droplevel()

In [14]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [15]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-07-30,17:00,Community Shield,FA Community Shield,Sat,Neutral,L,1.0,3.0,Liverpool,...,4-3-3,Craig Pawson,Match Report,NaN,14,8,NaN,NaN,0,0
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,...,4-3-3,Michael Oliver,Match Report,NaN,13,1,18.7,1.0,1,1
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,...,4-2-3-1,David Coote,Match Report,NaN,19,7,17.5,0.0,0,0
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,...,4-3-3,Jarred Gillett,Match Report,NaN,21,10,16.2,1.0,0,0
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,...,4-2-3-1,Darren England,Match Report,NaN,18,5,14.1,0.0,0,0


In [16]:
years = list(range(2023, 2019, -1))
all_matches = []

In [17]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [18]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(10)

In [19]:
len(all_matches)

80

In [20]:
match_df = pd.concat(all_matches)

In [21]:
match_df.columns = [c.lower() for c in match_df.columns]

In [22]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,...,Match Report,NaN,13.0,1.0,18.7,1.0,1.0,1.0,2023,Manchester City
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,...,Match Report,NaN,19.0,7.0,17.5,0.0,0.0,0.0,2023,Manchester City
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,...,Match Report,NaN,21.0,10.0,16.2,1.0,0.0,0.0,2023,Manchester City
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,...,Match Report,NaN,18.0,5.0,14.1,0.0,0.0,0.0,2023,Manchester City
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6.0,0.0,Nott'ham Forest,...,Match Report,NaN,17.0,9.0,14.8,0.0,0.0,0.0,2023,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2020-07-07,18:00,Premier League,Matchweek 34,Tue,Away,L,1,2,Watford,...,Match Report,NaN,12.0,3.0,16.9,1.0,0.0,0.0,2020,Norwich City
39,2020-07-11,12:30,Premier League,Matchweek 35,Sat,Home,L,0,4,West Ham,...,Match Report,NaN,11.0,2.0,19.8,1.0,0.0,0.0,2020,Norwich City
40,2020-07-14,20:15,Premier League,Matchweek 36,Tue,Away,L,0,1,Chelsea,...,Match Report,NaN,2.0,0.0,20.6,0.0,0.0,0.0,2020,Norwich City
41,2020-07-18,17:30,Premier League,Matchweek 37,Sat,Home,L,0,2,Burnley,...,Match Report,NaN,6.0,2.0,13.4,0.0,0.0,0.0,2020,Norwich City


In [23]:
match_df.to_csv("matches_2019_2023.csv")